In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Von_Hippel_Lindau"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Von_Hippel_Lindau/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Von_Hippel_Lindau/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Von_Hippel_Lindau/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Von_Hippel_Lindau/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# Step 1: Find the directory corresponding to Pheochromocytoma_and_Paraganglioma
import os

# List all directories in TCGA root directory
tcga_dirs = os.listdir(tcga_root_dir)

# Find the directory that matches our trait: Pheochromocytoma_and_Paraganglioma
matching_dirs = [dir_name for dir_name in tcga_dirs 
                if "pheochromocytoma" in dir_name.lower() or "paraganglioma" in dir_name.lower()]

if not matching_dirs:
    print(f"No matching directory found for trait: {trait}")
    # Record that this trait is not available and exit
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=False,
        is_trait_available=False
    )
else:
    # Select the most relevant directory
    selected_dir = matching_dirs[0]  # Should be 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)'
    cohort_dir = os.path.join(tcga_root_dir, selected_dir)
    
    # Step 2: Get file paths for clinical and genetic data
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    # Step 3: Load the files
    clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
    genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)
    
    # Step 4: Print column names of clinical data
    print("Clinical data columns:")
    print(clinical_df.columns.tolist())


Clinical data columns:
['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'age_at_initial_pathologic_diagnosis', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'ct_scan', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'disease_detected_on_screening', 'eastern_cancer_oncology_group', 'form_completion_date', 'gender', 'histological_type', 'history_of_neoadjuvant_treatment', 'history_pheo_or_para_anatomic_site', 'history_pheo_or_para_include_benign', 'icd_10', 'icd_o_3_histology', 'icd_o_3_site', 'informed_consent_verified', 'initial_weight', 'is_ffpe', 'karnofsky_performance_score', 'laterality', 'lost_follow_up', 'lymph_node_examined_count', 'new_neoplasm_confirmed_diagnosis_method_name', 'new_neoplasm_event_occurrence_anatomic_site', 'new_neoplasm_event_type', 'new_neoplasm_occurrence_anatomic_site_text', 'new_tumor

### Step 2: Find Candidate Demographic Features

In [3]:
# 1. Identify columns related to age and gender
candidate_age_cols = ["age_at_initial_pathologic_diagnosis", "days_to_birth"]
candidate_gender_cols = ["gender"]

# 2. Check available cohorts and find an appropriate one
import os

available_cohorts = os.listdir(tcga_root_dir)
print(f"Available cohorts: {available_cohorts}")

# Since we need to find data for Von Hippel-Lindau syndrome, we'll check each cohort
# First, ensure the output directories exist
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)

# Look for kidney-related cohorts as Von Hippel-Lindau commonly affects kidneys
kidney_cohorts = [cohort for cohort in available_cohorts if "KIRC" in cohort or "KIRP" in cohort or "KICH" in cohort]
if kidney_cohorts:
    cohort_dir = os.path.join(tcga_root_dir, kidney_cohorts[0])
else:
    # If no kidney cohorts, use the first available cohort
    cohort_dir = os.path.join(tcga_root_dir, available_cohorts[0])

print(f"Using cohort: {os.path.basename(cohort_dir)}")

# Load the clinical data
try:
    clinical_file_path, _ = tcga_get_relevant_filepaths(cohort_dir)
    clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
    
    # Extract age columns
    age_preview = {}
    for col in candidate_age_cols:
        if col in clinical_df.columns:
            age_preview[col] = clinical_df[col].head(5).tolist()
    
    # Extract gender columns
    gender_preview = {}
    for col in candidate_gender_cols:
        if col in clinical_df.columns:
            gender_preview[col] = clinical_df[col].head(5).tolist()
    
    print("Age columns preview:")
    print(age_preview)
    print("\nGender columns preview:")
    print(gender_preview)
    
except Exception as e:
    print(f"Error loading clinical data: {e}")
    # Create empty previews if there's an error
    age_preview = {}
    gender_preview = {}


Available cohorts: ['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)', 'TCGA_Colon_Cancer_(COAD)', 'TCGA_Endometrioid_Cancer_(UCEC)', 

### Step 3: Select Demographic Features

In [4]:
# 1. Choose the appropriate columns for age and gender

# For age, we have two candidate columns:
# - 'age_at_initial_pathologic_diagnosis': Contains direct age values
# - 'days_to_birth': Contains negative numbers representing days from birth to diagnosis

# 'age_at_initial_pathologic_diagnosis' is already in years and more directly interpretable
age_col = 'age_at_initial_pathologic_diagnosis'

# For gender, we only have one candidate column 'gender'
# The values look appropriate (MALE, FEMALE)
gender_col = 'gender'

# 2. Print out information about the chosen columns
print(f"Selected age column: {age_col}")
print(f"Age values sample: [53.0, 71.0, 71.0, 67.0, 80.0]")

print(f"Selected gender column: {gender_col}")
print(f"Gender values sample: ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']")


Selected age column: age_at_initial_pathologic_diagnosis
Age values sample: [53.0, 71.0, 71.0, 67.0, 80.0]
Selected gender column: gender
Gender values sample: ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize clinical features
# Find matching directory for Pheochromocytoma_and_Paraganglioma
matching_dirs = [dir_name for dir_name in os.listdir(tcga_root_dir) 
                if "pheochromocytoma" in dir_name.lower() or "paraganglioma" in dir_name.lower()]
selected_dir = matching_dirs[0]  # Should find 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)'
cohort_dir = os.path.join(tcga_root_dir, selected_dir)

# Get the file paths for clinical and genetic data
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

# Load the data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)

# Extract standardized clinical features using the provided trait variable
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait=trait,  # Using the provided trait variable
    age_col=age_col, 
    gender_col=gender_col
)

# Save the clinical data to out_clinical_data_file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Saved clinical data with {len(clinical_features)} samples")

# Step 2: Normalize gene symbols in gene expression data
# Transpose to get genes as rows
gene_df = genetic_df

# Normalize gene symbols using NCBI Gene database synonyms
normalized_gene_df = normalize_gene_symbols_in_index(gene_df)
print(f"After normalization: {len(normalized_gene_df)} genes remaining")

# Save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Saved normalized gene expression data")

# Step 3: Link clinical and genetic data
# Merge clinical features with genetic expression data
linked_data = clinical_features.join(normalized_gene_df.T, how='inner')
print(f"Linked data shape: {linked_data.shape} (samples x features)")

# Step 4: Handle missing values
cleaned_data = handle_missing_values(linked_data, trait_col=trait)
print(f"After handling missing values, data shape: {cleaned_data.shape}")

# Step 5: Determine if trait or demographics are severely biased
trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait=trait)

# Step 6: Validate data quality and save cohort information
note = "The dataset contains gene expression data along with clinical information for pheochromocytoma and paraganglioma patients."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=trait_biased,
    df=cleaned_data,
    note=note
)

# Step 7: Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Saved usable linked data to {out_data_file}")
else:
    print("Dataset was determined to be unusable and was not saved.")

Saved clinical data with 187 samples
After normalization: 19848 genes remaining


Saved normalized gene expression data
Linked data shape: (187, 19851) (samples x features)


After handling missing values, data shape: (187, 19851)
For the feature 'Von_Hippel_Lindau', the least common label is '0' with 3 occurrences. This represents 1.60% of the dataset.
The distribution of the feature 'Von_Hippel_Lindau' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 35.0
  50% (Median): 46.0
  75%: 57.5
Min: 19
Max: 83
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1' with 84 occurrences. This represents 44.92% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Dataset was determined to be unusable and was not saved.
